# Jour 11 - Olist x Freeglisse scraping case

<br>
<center>
<img src="https://res-2.cloudinary.com/crunchbase-production/image/upload/c_lpad,f_auto,q_auto:eco/h8ccwg5vhjcoqu1qfuwv" style="display:inline-block;" width=190>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img alt="Logo Freeglisse" src="https://freeglisse.com/img/ski-destock-logo-1569515072.jpg" width=250 style="display:inline-block;" >

## Case study: étude de l'offre et du positionnement de Freeglisse



**_Contexte_**

Olist aimerait désormais **augmenter son offre sur le segment du sport**, en incitant des marketplaces d'articles de sport à rejoindre la plateforme. Pour affiner sa stratégie vis-à-vis de ce type de vendeurs, Olist a besoin **d'analyser leur offre et leur positionnement**, pour mieux connaitre le marché sur lequel ils évoluent.

En particulier, les Sales d'Olist s'intéressent à **FreeGlisse**, une plateforme de vente en ligne de matériel de ski d'occasion. Leur ojectif est d'emmagasiner le plus d’informations possibles à partir de ce que FreeGlisse affiche sur son site web.
<br><br>

**_Instructions_**

Olist voudrait se concentrer dans un premier temps sur le segment des **skis**. Les informations à étudier incluent : **le nombre d'annonces sur les skis**, les **prix** pratiqués, la **répartition de l’offre entre les différentes sous-catégories**, eventuellement la **profondeur des stocks** (= le nombre de produits disponibles, ou le nombre de tailles disponibles par produit), et tout ce que vous pensez qui serait **intéressant pour analyser le positionnement marché de FreeGlisse**.

Ces informations seraient d’autant plus intéressantes si elles étaient étudiées **par type de ski** (indiqué dans la Fiche Technique de chaque ski). N'hésitez pas à effectuer les regroupements qui vous semblent pertinents, à faire des analyses quantitatives et qualitatives, en utilisant **pandas** et en faisant de beaux **graphiques**. Pour les graphiques, vous utiliserez **Google Data Studio**.

Côté technique, 2 compétences pourront être mises en oeuvre: le **crawling** entre les différentes pages du site, pour observer l’ensemble des offres de skis d’occasion (ici, pas besoin de selenium - des techniques plus simples seront possibles), et le **scraping des pages individuelles**, avec différentes infos à aller chercher.

Pour finir, l'objectif de ce cas est aussi de tirer de ces analyses des **insights business**, en particulier des **conclusions sur le positionnement commercial de FreeGlisse et les spécificités qu'il pourrait apporter à Olist**.
<br><br>

**_Présentation_**

En groupe, vous devrez **analyser les informations issues du scraping**, puis en fin de projet **présenter en environ 5 minutes** votre analyse à des Product Managers d'Olist (jurys de Databirdies), sous forme de **dashboard Data Studio**. L'essentiel est que la présentation soit **visuelle et intelligible par des stakeholders non-tech**.

## liste de nos idées 

In [ ]:
## - catégorie(performant, loisir, junior(mixte - fille), type(polyvalent, all mountain, piste...), )
# - taille de ski 
# - Etat matériel ( A B C) 
# - prix 
# - marque (moyen vente, moyen prix ...)


## importation des package

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from ydata_profiling import ProfileReport
import missingno as msno #msno.matrix(df)
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm


## Obtention des URL de chaque page

In [ ]:
def get_all_pages():
    urls_page = []
    page_number = 1
 
    for i in range(7):
        i = f"https://freeglisse.com/fr/12-ski-occasion?page={page_number}"
        page_number += 1
        urls_page.append(i)
    print(urls_page)
get_all_pages()

## Recuperation du code HTML de la page 1

In [ ]:
test = requests.get("https://freeglisse.com/fr/12-ski-occasion?page=1")
test.content


test.content est un objet bytes (un objet binaire)

In [ ]:
type(test.content)

## Mise en soup (meilleure lecture) du "test.content"

In [ ]:
soup = BeautifulSoup(test.content)
soup

## Ciblage sur la balise "h2" et "class":"h3 product-title"

In [ ]:
link = soup.find('h2', {"class":"h3 product-title"}) 
link

## Ciblage sur la balise "a" et extration du lien "href"

In [ ]:
link.a.get('href')

seconde méthode

In [ ]:
link.find('a', href = True )['href']

## Mise en place d'une boucle FOR pour extraire tout les url des articles (tout les liens "href")

In [ ]:
article_link = []
for link in soup.find_all('h2', {"class":"h3 product-title"}):
    article_link.append(link.a.get('href'))

résultat de la boucle

In [ ]:
article_link

In [ ]:
len(article_link)

il y a bien 24 articles par pages. La len de "article_link" nous prouve que la boucle a bien fonctionner

## Passage a l'obtention de tout les url (d'article) de chaque page

Reutilisation de morceau de code existant

In [ ]:
temporary_link = []
all_article_link = []
page_number = 1
for i in range(7):
    i = f"https://freeglisse.com/fr/12-ski-occasion?page={page_number}"
    test = requests.get(i)
    test.content
    soup = BeautifulSoup(test.content)
    for link in soup.find_all('h2', {"class":"h3 product-title"}):
        link.a.get('href')
        temporary_link.append(link.a.get('href'))
    all_article_link.extend(temporary_link)
    page_number += 1
    temporary_link = []


Résultat de la boucle

In [ ]:
all_article_link

In [ ]:
len(all_article_link)

La boucle fonctionne bien il y a bien 158 articles au moment de l'éxecution

## Extraction du détail produit 

In [ ]:
first_article = requests.get("https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm")
first_article.content

## mise en soup

In [ ]:
soup_first_article = BeautifulSoup(first_article.content)
soup_first_article

## ciblage sur la balise principal

In [ ]:
link_first_article = soup_first_article.find('div', {"class":"tab-content"}) 
link_first_article

## application du premier filtre 

les valeurs si dessous sont les informations du "détail produit"

In [ ]:
filter_one = link_first_article.find('section', {"class":"product-features"})
filter_one

Extraction du text du première article

In [ ]:
str(filter_one).split('Type')[1].split('>')[2].split('<')[0]
str(filter_one).split('Utilisateur')[1].split('>')[2].split('<')[0]
str(filter_one).split('Coloris')[1].split('>')[2].split('<')[0]

## Extraction du prix de tout les articles

In [ ]:
price = requests.get("https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm")
price.content

In [ ]:
soup_price = BeautifulSoup(price.content)
soup_price

In [ ]:
filter_price = soup_price.find('span', {"class":"current-price-value"}) 
filter_price

In [ ]:
str(filter_price).split('content')[1].split('\"')[1]

## Extraction de la marque

In [ ]:
maker = requests.get("https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm")
maker.content

In [ ]:
maker_soup = BeautifulSoup(maker.content)
maker_soup

In [ ]:
filter_maker = maker_soup.find('img',{'class':"img img-thumbnail manufacturer-logo"})
filter_maker

In [ ]:
filter_maker.get('alt')

## Mise en place du DataFrame de qualiter A

In [ ]:
from unidecode import unidecode
def table_qualite_a():
    base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page={}"
    current_page = 1
    df_global_a = pd.DataFrame(columns=['nom', 'prix', 'lien'])
    while True:
        url = base_url.format(current_page)
        response = requests.get(url)
        if response.status_code != 200:
            print("Erreur lors de la requête HTTP")
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
        noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]
        prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]
        product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]
        data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
        df_current = pd.DataFrame(data)
        df_global_a = pd.concat([df_global_a, df_current], ignore_index=True)
        next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
        if not next_page_link:
            break
        current_page += 1
    return df_global_a
df_a = table_qualite_a()
df_a['Qualité'] = 'Qualité A'
df_a

## Mise en place du DataFrame de qualiter B

In [ ]:
from unidecode import unidecode
def table_qualite_b():
    base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_b?page={}"
    current_page = 1
    df_global_b = pd.DataFrame(columns=['nom', 'prix', 'lien'])
    while True:
        url = base_url.format(current_page)
        response = requests.get(url)
        if response.status_code != 200:
            print("Erreur lors de la requête HTTP")
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
        noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]
        prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]
        product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]
        data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
        df_current = pd.DataFrame(data)
        df_global_b = pd.concat([df_global_b, df_current], ignore_index=True)
        next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
        if not next_page_link:
            break
        current_page += 1
    return df_global_b
df_b = table_qualite_b()
df_b['Qualité'] = 'Qualité B'
df_b

## Mise en place du DataFrame de qualiter C

In [ ]:
from unidecode import unidecode
def table_qualite_c():
    base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_c?page={}"
    current_page = 1
    df_global_c = pd.DataFrame(columns=['nom', 'prix', 'lien'])
    while True:
        url = base_url.format(current_page)
        response = requests.get(url)
        if response.status_code != 200:
            print("Erreur lors de la requête HTTP")
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
        noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]
        prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]
        product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]
        data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
        df_current = pd.DataFrame(data)
        df_global_c = pd.concat([df_global_c, df_current], ignore_index=True)
        next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
        if not next_page_link:
            break
        current_page += 1
    return df_global_c
df_c = table_qualite_c()
df_c['Qualité'] = 'Qualité C'
df_c

## Merge des 3 DataFrame de qualiter

In [ ]:
df_all_quality = pd.concat([df_a, df_b, df_c])
df_all_quality

## Mise en place du DataFrame

In [ ]:
def detail_produit():
    type_list = []
    price_list = []
    maker_list = []
    utilisateur_list = []
    niveau_list = []
    coloris_list = []
    utilisateur_config_list = []
    economie_co2_list = []
    type_produit_list = []
    for url in df_all_quality['lien']:
        s = requests.Session()
        r = s.get(url)
        product = BeautifulSoup(r.content)
        soup_price = BeautifulSoup(r.content)
        maker_soup = BeautifulSoup(r.content)
        product_info = product.find('section', {"class":"product-features"})
        filter_price = soup_price.find('span', {"class":"current-price-value"})
        filter_maker = maker_soup.find('img',{'class':"img img-thumbnail manufacturer-logo"})
        try :
            type_list.append(str(product_info).split('Type')[1].split('>')[2].split('<')[0])
        except :
            type_list.append(np.nan)
        try :
            price_list.append(str(filter_price).split('content')[1].split('\"')[1])
        except :
            price_list.append(np.nan)
        try :
            maker_list.append(filter_maker.get('alt'))
        except :
            maker_list.append(np.nan)
        try :
            utilisateur_list.append(str(product_info).split('Utilisateur')[1].split('>')[2].split('<')[0])
        except :
            utilisateur_list.append(np.nan)
        try :
            niveau_list.append(str(product_info).split('Niveau')[1].split('>')[2].split('<')[0])
        except :
            niveau_list.append(np.nan)
        try :
            coloris_list.append(str(product_info).split('Coloris')[1].split('>')[2].split('<')[0])
        except :
            coloris_list.append(np.nan)
        try :
            utilisateur_config_list.append(str(product_info).split('Utilisateur - Configurateur')[1].split('>')[2].split('<')[0])
        except :
            utilisateur_config_list.append(np.nan)
        try :
            economie_co2_list.append(str(product_info).split('Economie de CO2 pour la planète (en kg)')[1].split('>')[2].split('<')[0])
        except :
            economie_co2_list.append(np.nan)
        try :
            type_produit_list.append(str(product_info).split('Type de produit')[1].split('>')[2].split('<')[0])
        except :
            type_produit_list.append(np.nan)
    #return pd.DataFrame({
        #'url' : url_list, 
        #'price' : price_list,
        #'maker' : maker_list,
        #'type' : type_list,
        #'utilisateur' : utilisateur_list,
        #'niveau' : niveau_list,
        #'coloris' : coloris_list,
        #'utilisateur config' : utilisateur_config_list,
        #'economie co2' : economie_co2_list,
        #'type produit' : type_produit_list
    #})
    df_all_quality['marque_produit'] = maker_list
    df_all_quality['type_pratique'] = utilisateur_config_list
    df_all_quality['utilisateur'] = utilisateur_list
    df_all_quality['niveau'] = niveau_list
    df_all_quality['colori'] = coloris_list
    df_all_quality['economie co2'] = economie_co2_list
    df_all_quality['type'] = type_list
    df_all_quality['type produit'] = type_produit_list
    

affichage du DataFrame

In [ ]:
detail_produit()

In [ ]:
df_all_quality

changement des valeurs text en valeurs numerique

In [ ]:
df_all_quality['economie co2'].nunique()

affichage des valeurs manquantes

## Bonus: compilation des données Freeglisse

Quelques mois plus tard, Olist vous rappelle. Un partenariat a été noué avec FreeGlisse, et les produits d'occasion du site seront désormais vendus sur la plateforme Olist!

En revanche, les équipes d'Olist n'ont pas le temps de gérer à la main le transfert des annonces d'un site à l'autre. Ils ont besoin de vous pour **rassembler toutes les informations sur les skis d’occasion dans un fichier unique (excel ou csv)**, et pour **enregistrer l’ensemble des photos**, afin de pouvoir rapidement créer des fiches produit sur Olist. Ils comptent sur vous pour leur présenter un dossier bien organisé, afin que le transfert vers leur plateforme soit le plus simple possible.

Si vous avez le temps de vous lancer dans ce projet, vous pouvez inclure une courte partie là-dessus (slide ou section de notebook) dans votre **présentation finale**, afin **d'expliquer aux équipes d'Olist comment l'information est organisée**.